1-imports e path

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().parents[1]
DATA_DIR = ROOT / "data"

INPUT_FILE  = DATA_DIR / "costs.csv"
OUTPUT_FILE = DATA_DIR / "costs_cleaned.csv"

print("Raiz:", ROOT)
print("Entrada:", INPUT_FILE)
print("Saída:", OUTPUT_FILE)


Raiz: c:\AmbienteProgrmacao\Projetos\treinamento2\avalensurance-bia
Entrada: c:\AmbienteProgrmacao\Projetos\treinamento2\avalensurance-bia\data\costs.csv
Saída: c:\AmbienteProgrmacao\Projetos\treinamento2\avalensurance-bia\data\costs_cleaned.csv


2-lendo

In [7]:
df = pd.read_csv(INPUT_FILE)
print("Dimensões originais:", df.shape)
display(df.head())


Dimensões originais: (64800, 11)


,cost_id,person_id,policy_id,record_id,visit_id,annual_medical_cost,annual_premium,monthly_premium,claims_count,avg_claim_amount,total_claims_paid
0,cost_id_49197,64470,policy_id_00037,record_id_15635,visit_id_13715,5845.93,921.51,76.79,1.0,3917.22,3917.22
1,cost_id_15209,13405,policy_id_00069,record_id_18609,visit_id_30336,9024.53,1292.94,107.74,NaN,NaN,0.00
2,cost_id_37383,26180,policy_id_00069,record_id_28693,visit_id_53039,NaN,348.85,NaN,NaN,439.60,439.60
3,cost_id_48770,37875,policy_id_00123,record_id_54086,visit_id_47919,5340.85,732.72,61.06,2.0,1921.08,3842.16
4,cost_id_61529,5498,policy_id_00003,record_id_11096,visit_id_56767,4189.75,612.22,51.02,2.0,1856.81,3713.62


3-Preenchendo vazios

In [8]:
cols_target = ["annual_medical_cost","annual_premium","monthly_premium","claims_count","avg_claim_amount","total_claims_paid"]
df[cols_target] = df[cols_target].replace("", np.nan)
df[cols_target] = df[cols_target].fillna("-1")
display(df[cols_target].isna().sum())


annual_medical_cost    0
annual_premium         0
monthly_premium        0
claims_count           0
avg_claim_amount       0
total_claims_paid      0
dtype: int64

4- Ordenação + x duplicadas

In [9]:
df = df.sort_values(by="person_id", ascending=True)
dup_count = df.duplicated(subset="person_id").sum()
print(f"person_id duplicados antes de remover: {dup_count}")
df = df.drop_duplicates(subset="person_id", keep="first").reset_index(drop=True)
print("Dimensões após deduplicação:", df.shape)
display(df.head())


person_id duplicados antes de remover: 0
Dimensões após deduplicação: (64800, 11)


,cost_id,person_id,policy_id,record_id,visit_id,annual_medical_cost,annual_premium,monthly_premium,claims_count,avg_claim_amount,total_claims_paid
0,cost_id_31150,1,policy_id_00033,record_id_36092,visit_id_40611,839.85,290.63,24.22,1.0,410.76,410.76
1,cost_id_61371,2,policy_id_00119,record_id_45363,visit_id_48818,709.72,278.13,23.18,1.0,-1,269.0
2,cost_id_63662,3,policy_id_00176,record_id_64734,visit_id_01246,1589.27,372.57,31.05,4.0,270.0,-1
3,cost_id_17436,4,policy_id_00112,record_id_43770,visit_id_47786,3246.81,677.54,56.46,0.0,0.0,0.0
4,cost_id_59155,6,policy_id_00017,record_id_61337,visit_id_35905,1462.55,350.4,29.2,3.0,219.58,658.74


5- exportação

In [10]:
df.to_csv(OUTPUT_FILE, index=False)
print(f"Arquivo salvo em: {OUTPUT_FILE}")


Arquivo salvo em: c:\AmbienteProgrmacao\Projetos\treinamento2\avalensurance-bia\data\costs_cleaned.csv
